In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [9]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (207.28 seconds): Collecting URLs... theverge            

n_total     25861
n_unique    25798
dtype: int64

,pubs,actual_pub
https://businessinsider.co.za/automobile-association-south-africa-vehicle-purchases-keeping-vehicles-for-longer-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/new-2020-coin-designs-natura-dinosaur-and-kruger-rand-2019-10,"businessinsider, news24",businessinsider
https://dailymail.co.uk/news/article-7561421/Trump-campaign-capitalizes-latest-target-selling-25-Wheres-Hunter-tees.html,"realclearpolitics, dailymail",dailymail
https://hotair.com/archives/ed-morrissey/2019/10/11/scotus-come-trump-loses-appeal-tax-records,"townhall, hotair",hotair
https://msnbc.com/hallie-jackson/watch/appeals-court-upholds-democrats-subpoena-for-trump-financial-records-71067717523,"msnbc, nbcnews",msnbc
https://msnbc.com/katy-tur/watch/court-rules-trump-violated-law-with-emergency-order-to-build-border-wall-71089221536,"msnbc, nbcnews",msnbc
https://msnbc.com/morning-joe/watch/gop-senator-trying-to-have-it-both-ways-says-fmr-rep-71061573866,"msnbc, nbcnews",msnbc
https://msnbc.com/stephanie-ruhle/watch/amb-sondland-set-to-testify-despite-state-department-direction-71061061990,"msnbc, nbcnews",msnbc
https://msnbc.com/the-beat-with-ari/watch/swizz-beatz-on-working-with-beyonce-jay-z-drake-and-upending-the-art-world-70266437589,"msnbc, nbcnews",msnbc
https://nationalreview.com/2019/10/turkey-and-the-kurds-its-more-complicated-than-you-think,"realclearpolitics, nationalreview",nationalreview


huffpost                  74
cnn                      836
investing.com           1021
politico                 218
time                      13
cnbc                     160
fox                      329
bbc                      277
businessinsider          594
morningstar              102
wsj                      111
nytimes                  112
guardian                 166
reuters                 1165
washingtontimes          466
washingtonpost           129
cbs                      402
marketwatch              179
atlantic                  95
vice                     104
npr                      863
newrepublic               20
yahoo                    399
independent              716
heritage                 280
zdnet                    388
townhall                 627
abcnews                  105
hotair                    84
cbc                      284
                        ... 
metro                    242
msnbc                    104
nationalreview           745
news24        

In [10]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [11]:
select_results = crawler.select(); display(*select_results)

100.00% (28.45 seconds): Selecting URLs... cryptonews          

n_total     4139
n_unique    4139
dtype: int64

,pubs,actual_pub


huffpost                 13
cnn                      86
investing.com           187
politico                 25
time                      3
cnbc                     29
fox                     120
bbc                      34
businessinsider         186
morningstar              10
wsj                      44
nytimes                  17
guardian                 61
reuters                 401
washingtontimes          71
washingtonpost           64
cbs                      38
marketwatch              53
atlantic                 12
vice                     35
npr                      13
newrepublic               3
yahoo                   229
independent              71
heritage                  1
zdnet                    23
townhall                 29
abcnews                  49
hotair                   37
cbc                      42
                       ... 
metro                   100
msnbc                    27
nationalreview           43
news24                  130
techcrunch          

In [12]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [13]:
download_results = crawler.download(); download_results

100.00% (4142.46 seconds): Downloading... dailymail           

,downloaded,trashed
abcnews,49,0
afr,25,0
aljazeera,19,78
americanconservative,7,2
arirang,0,0
arstechnica,7,0
arynews,15,27
atlantic,12,0
axios,36,28
bbc,28,6


In [14]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

[]